In [1]:
#!pip install torch

In [2]:
#!pip install import_ipynb

In [3]:
#!pip install torchvision

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
import pandas as pd
import numpy as np
import torch as T
import torch
import math
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import import_ipynb
from model import *
import random
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from pickle import dump,load
#from helper_functions import *
#from dataset import get_data_loader
import torchvision.utils as utils
import argparse
from torch.autograd import Variable
from argparse import ArgumentParser
import matplotlib.pyplot as plt
%matplotlib inline

save_path = "data/chil/model.tar"
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# if not os.path.exists("data/saved_models"):
#     os.makedirs("data/saved_models")

importing Jupyter notebook from oldModel2.ipynb


In [5]:
ARG_PARSER = ArgumentParser()

ARG_PARSER.add_argument('--num_epochs', default=100, type=int)
ARG_PARSER.add_argument('--seq_len', default=40, type=int)
ARG_PARSER.add_argument('--pred_len', default=8, type=int)
ARG_PARSER.add_argument('--missingRate', default=10, type=int)
ARG_PARSER.add_argument('--batch_size', default=197, type=int)
ARG_PARSER.add_argument('--patience', default=20, type=int)
ARG_PARSER.add_argument('--resume_training', default=False)
ARG_PARSER.add_argument('--fine_tune', default=False)
ARG_PARSER.add_argument('--train', default=False)
ARG_PARSER.add_argument('--evalPred', default=True)
ARG_PARSER.add_argument('--predWin', default=28, type=int)
ARG_PARSER.add_argument('--discriminator', default=True)
ARG_PARSER.add_argument('--noise', default=False)
ARG_PARSER.add_argument('--attn', default=True)
ARG_PARSER.add_argument('--female', default=True)
ARG_PARSER.add_argument('--male', default=False)
ARG_PARSER.add_argument('--binary', default=True)

_StoreAction(option_strings=['--binary'], dest='binary', nargs=None, const=None, default=True, type=None, choices=None, help=None, metavar=None)

In [6]:
ARGS = ARG_PARSER.parse_args(args=[])

if ARGS.female:
    INPUT_DIM = 662
    DRUG_DIM=1279
    GENDER = "female"
    G = "F"
if ARGS.male:
    INPUT_DIM = 610
    DRUG_DIM=1097
    GENDER = "male"
    G = "M"
    
#INPUT_DIM = 662#Male = 610,Female=662
OUTPUT_DIM = 2
AGE_DIM = 1
#DRUG_DIM = 1279#Male = 1097,Female=1279
ENC_EMB_DIM = 52
DEC_EMB_DIM = 12
ENC_HID_DIM = 52
DEC_HID_DIM = 32
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
D_HID_SIZE = 10
MAX_SEQ_LEN = ARGS.seq_len
BATCH_SIZE = ARGS.batch_size
EPSILON = 1e-40



In [7]:
%matplotlib inline
def plot_grad_flow(named_parameters):
    ave_grads = []
    layers = []
    for n, p in named_parameters:
        if ((n != 'encoder.ageEmbedding.W_emb') & ((n != 'decoder.ageEmbedding.W_emb'))):
            if(p.requires_grad) and ("bias" not in n):
                layers.append(n)
                ave_grads.append(p.grad.abs().mean())
    #print(layers)
    #print(ave_grads)
    plt.plot(ave_grads, alpha=0.3, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(xmin=0, xmax=len(ave_grads))
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.show()

In [ ]:
def testAnalysis(oBmi,iBmi,oAge,oPids,oAgeIndex,oAttn,oAttnIdx):
    forecast=torch.stack(oBmi)
    labels=torch.stack(iBmi)
    forecast=forecast.argmax(1)
    prob=torch.stack(oBmi)
    forecast=forecast.cpu()
    labels=labels.cpu()
    prob =prob.cpu()
    prob = torch.nn.functional.softmax(prob,dim=1)
    y_true=labels
    
    acc = accuracy_score(labels, forecast)
    precision = precision_score(labels, forecast)
    recall = recall_score(labels, forecast)
    print(acc,precision,recall)
    fpr, tpr, thresholds = roc_curve(labels, prob[:,1])
    apr = average_precision_score(labels, prob[:,1])
    print("Total samples ",len(labels))
    pos = (labels.sum()).item()
    print("Total positives ",pos)
    print("Baseline ",(pos)/len(labels))
    print("auc",auc(fpr, tpr))
    print("apr",apr)

In [8]:
def pred_test(args, model,train_on_gpu):
    model['g'].eval()
    
    RLoss=0
    TBatches=0
    oBmi=[]
    iBmi=[]
    oAge=[]
    oPids=[]
    oAttn=[]
    oAttnIdx=[]
    oAgeIndex=[]

    with T.autograd.no_grad():
        files = 'cond'+G+'test.csv'
        
        drugFiles = 'drug'+G+'test.csv'
    
        maskFiles = 'mask'+G+'test.csv'
        
        dataset = CSVDataset(files, int(args.seq_len*BATCH_SIZE),1356100,args.seq_len,flag=0)
        drugDataset = CSVDataset(drugFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=2)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
        drugLoader = DataLoader(drugDataset,batch_size=1,num_workers=0, shuffle=False)
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        
        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, drugLoader, maskLoader)):
            data,drug,mask=allData
            pids=data[1]
            data=data[0]
            if args.female:
                age=data[:,:,:,663]
                data=data[:,:,:,1:663]    
                drug=drug[:,:,:,1:1280]
            if args.male:
                age=data[:,:,:,611]#Female=241, Male = 346]
                data=data[:,:,:,1:611]#Female = 1:229, Male = 1:334]
                drug=drug[:,:,:,1:1098]
            target=mask[:,:,:,:4]
            mask=mask[:,:,:,4]
            

            age=age.squeeze()
            data=data.squeeze()
            pids=pids.squeeze()
            drug=drug.squeeze()
            mask=mask.squeeze()
            target=target.squeeze()
            
            age=age.unsqueeze(2)
            
            ########====BINARY=====##########
            if args.binary:
                    target[:,:,0] = target[:,:,0]+target[:,:,1]+target[:,:,2]
                    target[:,:,1] = target[:,:,3]
                    target=target[:,:,0:2]
            
            pids=pids.squeeze()
            pids=pids.view(data.shape[0],data.shape[1])
#             print("pids",pids.shape)
#             print("pids",pids[0])
            testMask = mask.clone().detach()
            y = target.clone().detach()
            #print("ageOutside",age[0])
        
             #------------remove last 5 timestamps------------------
            #print(data[0:10,8:,653])
#             print("=====Initial Data===============")
#             #print("data",data[0,0,:])
#             print("mask",mask[0])
#             print("testMask",testMask[0])
            
            for i in range(data.shape[0]):
                #total length
                j=40
                #prediction window
                k=args.predWin
                #remove values from pred window
                data[i,j-k:j,:]=0
                drug[i,j-k:j,:]=0
                mask[i,j-k:j]=0
                y[i,j-k:j,:]=0
                #zero values in observation window, so that it can be used to check loss in pred win only
                target[i,0:j-k,:]=0
                testMask[i,0:j-k]=0
            
#             print("=====test Data===============")
#             #print("data",data[0,0,:])
#             print("mask",mask[0])
#             print("testMask",testMask[0])
            
#             print("=====idxs Data===============")
            #check where obs win is empty
            emptyIds=torch.sum(data.reshape(data.shape[0],-1),dim=1)
#             print("emptyIds",emptyIds.shape)
#             print("emptyIds",emptyIds)
            idxs = torch.nonzero(emptyIds > 0)
            idxs=idxs.squeeze()
            idxs=idxs.view(-1)
#             print("idxs",idxs.shape)
            #get only data points that have atleast 1 entry in obs window
            data=data[idxs]
            drug=drug[idxs]
            mask=mask[idxs]
            y=y[idxs]
            target=target[idxs]
            testMask=testMask[idxs]
            pids=pids[idxs]
            age=age[idxs]
            
#             print("data",data.shape)
#             print("drug",drug.shape)
#             print("mask",mask.shape)
#             print("target",target.shape)
#             print("pids",pids.shape)
#             print("testMask",testMask.shape)
#             print("age",age.shape)

             #------------E training------------------
            #data=torch.cat((data,y),2)
            if train_on_gpu:
                data=data.cuda(0)
                drug=drug.cuda(0)
                target=target.cuda(0)
                age=age.cuda(0)
                mask=mask.cuda(0)
                y=y.cuda(0)
                testMask=testMask.cuda(0)
            #output = model['g'](data, drug, y, age, mask, args.train, train_on_gpu)[0]
            if args.attn:
                output,attn = model['g'](data, drug, y, age, mask, args.train, train_on_gpu)
            else:
                output = model['g'](data, drug,  y, age, mask, args.train, train_on_gpu)

            #------------R Loss------------------
            
            if args.attn:
                attn = attn.permute(1,0,2)
                attn = attn.reshape(-1,attn.shape[2])
#                 print("attn",attn.shape)
                #values, indices = torch.topk(attn,k=10,dim=1)
                values = attn[:,0:12]
                indices = torch.arange(0,12)
                indices = indices.unsqueeze(0)
                indices = indices.repeat(values.shape[0],1)

            output = output.permute(1,0,2)
            #print("output",output.shape)
            output = output.reshape(-1,output.shape[2])
#             print("output",output.shape)
            #print("target",target.shape)
            ##print("output",output[0,0,:])
            ##print("target",target[0,0,:])
            top1 = target.argmax(2)
            top1=top1.view(-1)
#             print("top1",top1.shape)
            

            testMask=testMask.view(-1)
            #print("mask",mask)
            
            #idxs are indices where mask is 1 i:e value is present to calculate supervised loss
            idxs = torch.nonzero(testMask == 1)
#             print("idxs",idxs.shape)
            idxs=idxs.squeeze()
#             idxs=idxs.view(-1)
            print("idxs",idxs.shape)
#             print("idxs",idxs)
            #print("mask",mask[idxs])
    
            #print(age.shape)

            pids=pids.view(-1)
            age=age.view(-1)
#             print(age.shape)
            ageIndices=torch.tensor(np.array(range(0,40)))
            ageIndices=ageIndices.repeat(200)
#             print(ageIndices.shape)
            if args.attn:
                values=values.cpu()
                indices=indices.cpu()

            ##print("output",output)
            ##print("top1",top1)
            
            if(idxs.shape[0]!=0):
                output = output[idxs,:]
                if args.attn:
                    values = values[idxs,:]
                    indices = indices[idxs,:]
                    values=values.cpu()
                    indices=indices.cpu()
                top1 = top1[idxs]
                age=age[idxs]
                ageIndices=ageIndices[idxs]
                pids=pids[idxs]
                
#                 print(age.shape)
#                 print(ageIndices.shape)
            
                loss = nn.CrossEntropyLoss()
                RLoss = RLoss + loss(output, top1)
                #print("loss",RLoss)

                #Preparing outputs for testing statistics
                oBmi.extend(output)
                iBmi.extend(top1)
                oAge.extend(age)
                oPids.extend(pids)
                oAgeIndex.extend(ageIndices)
                if args.attn:
                    oAttn.extend(values)
                    oAttnIdx.extend(indices)
            #print("oBmi",len(oBmi))
            #print("oBmi",oBmi[0])
            #print("iBmi",len(iBmi))
            #print("oAge",len(oAge))
            #print("oPids",len(oPids))
            
        TBatches=TBatches+batch_idx+1
        RLoss = RLoss/TBatches
    
    #print("===================================")
    #oBmi=np.asarray(oBmi)
    #iBmi=np.asarray(iBmi)
    #oAge=np.asarray(oAge)
    #iBmi=np.asarray(iBmi)
    
    print("RLoss:",RLoss)
    #print("MAE Loss Forward:",mseLossF)
    #print(outputBMI)
    testAnalysis(oBmi,iBmi,oAge,oPids,oAgeIndex,oAttn,oAttnIdx)
    return oBmi,iBmi,oAge,oPids,oAgeIndex,oAttn,oAttnIdx
                
                

In [9]:
def run_evalFull(args, model,train_on_gpu):
    model['g'].eval()
    
    RLoss=0
    TBatches=0
    
    with T.autograd.no_grad():
        files = 'cond'+G+'val.csv'
        
        drugFiles = 'drug'+G+'val.csv'
    
        maskFiles = 'mask'+G+'val.csv'
        #print(files)
        
        dataset = CSVDataset(files, int(args.seq_len*BATCH_SIZE),1356100,args.seq_len,flag=0)
        drugDataset = CSVDataset(drugFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)
        maskDataset = CSVDataset(maskFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)

        loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
        drugLoader = DataLoader(drugDataset,batch_size=1,num_workers=0, shuffle=False)
        maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)

        
        loss={}

        #for every batch
        for batch_idx, allData in enumerate(zip(loader, drugLoader, maskLoader)):
            data,drug,mask=allData
            pids=data[1]
            data=data[0]
            #print(data.shape)
            #print(mask.shape)
            if args.female:
                    age=data[:,:,:,663]
                    data=data[:,:,:,1:663]    
                    drug=drug[:,:,:,1:1280]
            if args.male:
                age=data[:,:,:,611]#Female=241, Male = 346]
                data=data[:,:,:,1:611]#Female = 1:229, Male = 1:334]
                drug=drug[:,:,:,1:1098]
            target=mask[:,:,:,:4]
            mask=mask[:,:,:,4]
            
            
            #print(age.shape)
            #print(data.shape)
            #print(mask.shape)
            #print(target.shape)
            
            age=age.squeeze()
            data=data.squeeze()
            pids=pids.squeeze()
            drug=drug.squeeze()
            mask=mask.squeeze()
            target=target.squeeze()
            age=age.unsqueeze(2)
            
            if args.binary:
                    target[:,:,0] = target[:,:,0]+target[:,:,1]+target[:,:,2]
                    target[:,:,1] = target[:,:,3]
                    target=target[:,:,0:2]
            
            if(args.fine_tune):
                #======================Fine tuning with obs/pred window task=============================
                pids=pids.view(data.shape[0],data.shape[1])
            #             print("pids",pids.shape)
            #             print("pids",pids[0])
                testMask = mask.clone().detach()
                y = target.clone().detach()
                #print("ageOutside",age[0])

                 #------------remove last 5 timestamps------------------
                #print(data[0:10,8:,653])
            #             print("=====Initial Data===============")
            #             #print("data",data[0,0,:])
            #             print("mask",mask[0])
            #             print("testMask",testMask[0])

                for i in range(data.shape[0]):
                    #total length
                    j=40
                    #prediction window
                    k=args.predWin
                    #remove values from pred window
                    data[i,j-k:j,:]=0
                    drug[i,j-k:j,:]=0
                    mask[i,j-k:j]=0
                    y[i,j-k:j,:]=0
                    #zero values in observation window, so that it can be used to check loss in pred win only
                    target[i,0:j-k,:]=0
                    testMask[i,0:j-k]=0

            #             print("=====test Data===============")
            #             #print("data",data[0,0,:])
            #             print("mask",mask[0])
            #             print("testMask",testMask[0])

            #             print("=====idxs Data===============")
                #check where obs win is empty
                emptyIds=torch.sum(data.reshape(data.shape[0],-1),dim=1)
            #             print("emptyIds",emptyIds.shape)
            #             print("emptyIds",emptyIds)
                idxs = torch.nonzero(emptyIds > 0)
                idxs=idxs.squeeze()
            #             print("idxs",idxs.shape)
                #get only data points that have atleast 1 entry in obs window
                data=data[idxs]
                drug=drug[idxs]
                mask=mask[idxs]
                y=y[idxs]
                target=target[idxs]
                testMask=testMask[idxs]
                pids=pids[idxs]
                age=age[idxs]

            #             print("data",data.shape)
            #             print("drug",drug.shape)
            #             print("mask",mask.shape)
            #             print("target",target.shape)
            #             print("pids",pids.shape)
            #             print("testMask",testMask.shape)
            #             print("age",age.shape)

                #data=torch.cat((data,y),2)
                if train_on_gpu:
                    y=y.cuda(0)
                    testMask=testMask.cuda(0)
                #========================================================================================
            
            #data=torch.cat((data,target),2)

            #------------E training------------------
            if train_on_gpu:
                data=data.cuda(0)
                target=target.cuda(0)
                age=age.cuda(0)
                drug=drug.cuda(0)
                mask=mask.cuda(0)
            output = model['g'](data, drug, target, age, mask, args.train,train_on_gpu)

            #------------R Loss------------------
            #print("output",output.shape)

            output = output.permute(1,0,2)
            #print("output",output.shape)
            output = output.reshape(-1,output.shape[2])
            #print("output",output.shape)
            #print("target",target.shape)
            ##print("output",output[0,0,:])
            ##print("target",target[0,0,:])
            top1 = target.argmax(2)
            top1=top1.view(-1)
            #print("top1",top1.shape)

            mask=mask.view(-1)
            if(args.fine_tune):
                testMask=testMask.view(-1)
                idxs = torch.nonzero(testMask == 1)
            else:
                idxs = torch.nonzero(mask == 1)
            idxs=idxs.squeeze()
            idxs=idxs.view(-1)
            #print("idxs",idxs.shape)

            output = output[idxs,:]
            top1 = top1[idxs]
            #print("output",output.shape)
            #print("top1",top1.shape)

            ##print("output",output)
            ##print("top1",top1)
            
            if(idxs.shape[0]!=0):
                #print("x")
                loss = nn.CrossEntropyLoss()
                RLoss = RLoss + loss(output, top1)
            #print("loss",RLoss)

        TBatches=TBatches+batch_idx+1
        #print(TBatches)
    #print(TBatches)
    RLoss = RLoss/TBatches
    #print("===================================")
    print("Val R Loss:",RLoss)
#     print("Val G Loss:",loss['g'])
#     print("Val D Loss:",DLoss/TBatches)
    return RLoss
                
                
                


In [10]:
%matplotlib inline
def batch_run(model, data, pids, drug, mask, age, target, args, optimizer,train_on_gpu):
    
    rloss=0
    dloss=0
    #print("==============going to encoder=====================")
    loss={}
    freeze_d=False
    #age=age.unsqueeze(2)
    ##print("age",age[0,:,:])
    #print(age.shape)
    #print(data.shape)
    #print(mask.shape)
    #print(target.shape)
    
    if(args.fine_tune):
        #======================Fine tuning with obs/pred window task=============================
        pids=pids.view(data.shape[0],data.shape[1])
    #             print("pids",pids.shape)
    #             print("pids",pids[0])
        testMask = mask.clone().detach()
        y = target.clone().detach()
        #print("ageOutside",age[0])

         #------------remove last 5 timestamps------------------
        #print(data[0:10,8:,653])
    #             print("=====Initial Data===============")
    #             #print("data",data[0,0,:])
    #             print("mask",mask[0])
    #             print("testMask",testMask[0])

        for i in range(data.shape[0]):
            #total length
            j=40
            #prediction window
            k=args.predWin
            #remove values from pred window
            data[i,j-k:j,:]=0
            drug[i,j-k:j,:]=0
            mask[i,j-k:j]=0
            y[i,j-k:j,:]=0
            #zero values in observation window, so that it can be used to check loss in pred win only
            target[i,0:j-k,:]=0
            testMask[i,0:j-k]=0

    #             print("=====test Data===============")
    #             #print("data",data[0,0,:])
    #             print("mask",mask[0])
    #             print("testMask",testMask[0])

    #             print("=====idxs Data===============")
        #check where obs win is empty
        emptyIds=torch.sum(data.reshape(data.shape[0],-1),dim=1)
    #             print("emptyIds",emptyIds.shape)
    #             print("emptyIds",emptyIds)
        idxs = torch.nonzero(emptyIds > 0)
        idxs=idxs.squeeze()
    #             print("idxs",idxs.shape)
        #get only data points that have atleast 1 entry in obs window
        data=data[idxs]
        drug=drug[idxs]
        mask=mask[idxs]
        y=y[idxs]
        target=target[idxs]
        testMask=testMask[idxs]
        pids=pids[idxs]
        age=age[idxs]

    #             print("data",data.shape)
    #             print("drug",drug.shape)
    #             print("mask",mask.shape)
    #             print("target",target.shape)
    #             print("pids",pids.shape)
    #             print("testMask",testMask.shape)
    #             print("age",age.shape)

        #data=torch.cat((data,y),2)
        if train_on_gpu:
            y=y.cuda(0)
            testMask=testMask.cuda(0)
        #========================================================================================
    
    if train_on_gpu:
        data=data.cuda(0)
        drug=drug.cuda(0)
        target=target.cuda(0)
        age=age.cuda(0)
        mask=mask.cuda(0)
    
    #------------E training------------------
    if  args.discriminator:
        optimizer['d'].zero_grad()
    optimizer['g'].zero_grad()
    output = model['g'](data, drug, target, age, mask, args.train, train_on_gpu)
    
    #print("==============Batch Over=============")
    #print("output",output.shape)
    
    output = output.permute(1,0,2)
    #print("output",output.shape)
    out = output.argmax(2)
    output = output.reshape(-1,output.shape[2])
    #print("output",output.shape)
    #print("target",target.shape)
    ##print("output",output[0,0,:])
    ##print("target",target[0,0,:])
    top1 = target.argmax(2)
    
    #------------D training------------------
    if  args.discriminator:
#         optimizer['d'].zero_grad()
        mask=mask.float()
        top1=top1.float()
        out=out.float()
        decodedOutput=out*(1-mask)+top1*(mask)
        #print("decodedOutput",decodedOutput.shape)
        #print("decodedOutput",decodedOutput[0,:])
        lossDf = model['d'](decodedOutput, mask, direct="forward")
        lossDb = model['d'](decodedOutput, mask, direct="backward")
        #print("================Outside Discriminator=================")
        dloss=lossDf['loss_d']+lossDb['loss_d']
        
        
    mask=mask.long()
    top1=top1.long()
    top1=top1.view(-1)
    #print("top1",top1.shape)
    
    mask=mask.view(-1)
    if(args.fine_tune):
        testMask=testMask.view(-1)
        idxs = torch.nonzero(testMask == 1)
    else:
        idxs = torch.nonzero(mask == 1)
    #print("mask",mask.shape)
    
    
    idxs=idxs.squeeze()
    idxs=idxs.view(-1)
    #print("idxs",idxs)
    #print("idxs",idxs.shape)
    
    output = output[idxs,:]
    top1 = top1[idxs]
    #print("output",output.shape)
    #print("top1",top1.shape))
    #print("output",output)
    #print("top1",top1)
    if(idxs.shape[0]!=0):
        #print("size zero")
        loss = nn.CrossEntropyLoss()
        rloss = loss(output, top1)
       # print("loss",rloss)
    
        if  args.discriminator:
            rloss1=rloss+lossDf['loss_g']+lossDb['loss_g']
            dloss.backward(retain_graph=True)
            rloss1.backward()
        else:
            rloss.backward()

        #------------E G training------------------
        #optimizer['e'].zero_grad()
        #optimizer['g'].zero_grad()
        #rloss.backward()
        #optimizer['e'].step()
        if  args.discriminator:
            optimizer['d'].step()
        optimizer['g'].step()
    
    
    
    if  args.discriminator:
        dloss=dloss.item()
        return rloss.item(),dloss
    else:
        return rloss.item()

In [11]:
def plotBmi(outBmi, inBmi, age):
    
    outBmi = outBmi.cpu().detach().numpy()
    inBmi = inBmi.cpu().detach().numpy()
    testMask = testMask.cpu().detach().numpy()
    
    
    outBmi=outBmi[np.nonzero(testMask)]
    inBmi=inBmi[np.nonzero(testMask)]
    
    
    return outBmi,inBmi

In [12]:
%matplotlib inline
def run_epoch(args, model, optimizer, train_on_gpu):
    ''' Run a single epoch
    '''
    
    decodedOutput=[]
    trainLoss=[]
    trainDLoss=[]
    valLoss=[]
    # initialize the early_stopping object
    early_stopping = EarlyStopping(args, patience=args.patience, verbose=True)
    if args.resume_training:
        early_stopping(0.3319, model, optimizer, save_path)
    #for evrey epoch
    for epoch in range(args.num_epochs):
        model['g'].train()
        if  args.discriminator:
            model['d'].train()

        ##print(model['e'])
        ##print(model['g'])
    
        #Running Losses
        RLoss=0
        DLoss=0
#         GLoss=0
        TBatches=0  
        #output=[]
        print("=============EPOCH================= ",epoch)
        freeze_d=False
        #if epoch<=args.g_pretraining_epochs:
            #freeze_d=True
       
        for i in [1]:
            
#             filesPos = 'cond'+G+'trainPos2'+str(i)+'.csv'

#             drugFilesPos = 'drug'+G+'trainPos2'+str(i)+'.csv'

#             maskFilesPos = 'mask'+G+'trainPos2'+str(i)+'.csv'
            
            files = 'cond'+G+'train'+'.csv'

            drugFiles = 'drug'+G+'train'+'.csv'

            maskFiles = 'mask'+G+'train'+'.csv'


            dataset = CSVDataset(files, int(args.seq_len*BATCH_SIZE),1356100,args.seq_len,flag=0)
            drugDataset = CSVDataset(drugFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)
            maskDataset = CSVDataset(maskFiles, int(args.seq_len*BATCH_SIZE),1356100, args.seq_len,flag=1)
            
#             datasetPos = CSVDataset(filesPos, int(args.seq_len*70),1356100,args.seq_len,flag=0)
#             drugDatasetPos = CSVDataset(drugFilesPos, int(args.seq_len*70),1356100, args.seq_len,flag=1)
#             maskDatasetPos = CSVDataset(maskFilesPos, int(args.seq_len*70),1356100, args.seq_len,flag=1)

            loader = DataLoader(dataset,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
            drugLoader = DataLoader(drugDataset,batch_size=1,num_workers=0, shuffle=False)
            maskLoader = DataLoader(maskDataset,batch_size=1,num_workers=0, shuffle=False)
            
#             loaderPos = DataLoader(datasetPos,batch_size=1,num_workers=0, shuffle=False)#number of times getitem is called in one iteration
#             drugLoaderPos = DataLoader(drugDatasetPos,batch_size=1,num_workers=0, shuffle=False)
#             maskLoaderPos = DataLoader(maskDatasetPos,batch_size=1,num_workers=0, shuffle=False)
            
            ##print(files)
            #for every batch
            for batch_idx, allData in enumerate(zip(loader, drugLoader, maskLoader)):
                #, loaderPos, drugLoaderPos, maskLoaderPos)):
                #bmi_norm=dataset.bmi_norm
    #             print('batch: {}'.format(batch_idx))
#                 data,drug,mask,dataPos,drugPos,maskPos=allData
                data,drug,mask=allData
                pids=data[1]
                data=data[0]
                
#                 print(data.shape)
#                 print(pids.shape)
#                 print(drug.shape)
#                 print(mask.shape)

#                 pidsPos=dataPos[1]
#                 dataPos=dataPos[0]
#                 data=torch.cat((data,dataPos),1)
#                 pids=torch.cat((pids,pidsPos),1)
#                 drug=torch.cat((drug,drugPos),1)
#                 mask=torch.cat((mask,maskPos),1)
                
#                 print(data.shape)
#                 print(pids.shape)
#                 print(drug.shape)
#                 print(mask.shape)
    #             print("Age",data[0,0,:,346])
                #print(mask.shape)
                #print("drug",drug.shape)
                if args.female:
                    age=data[:,:,:,663]
                    data=data[:,:,:,1:663]    
                    drug=drug[:,:,:,1:1280]
                if args.male:
                    age=data[:,:,:,611]
                    data=data[:,:,:,1:611]
                    drug=drug[:,:,:,1:1098]
                target=mask[:,:,:,:4]
                mask=mask[:,:,:,4]



                age=age.squeeze()
                data=data.squeeze()
                pids=pids.squeeze()
                drug=drug.squeeze()
                mask=mask.squeeze()
                target=target.squeeze()

                age=age.unsqueeze(2)
    #             print(age.shape)
    #             print(data.shape)
    #             print(mask.shape)
#                 print(target.shape)
#                 print(target[22,:,:])
        
                ########====BINARY=====##########
                if args.binary:
                    target[:,:,0] = target[:,:,0]+target[:,:,1]+target[:,:,2]
                    target[:,:,1] = target[:,:,3]
                    target=target[:,:,0:2]
#                 print(target[22,:,:])
#                 print(target.shape)

               # data=torch.cat((data,target),2)

    #             print("age",age[0])
                #print("data",data.shape)
                #print("mask",mask[0,:])
                #print("target",target[0,:,:])

                ##print("age",age[0,:])

                if  args.discriminator:
                        rloss,dloss=batch_run(model, data, pids, drug,mask, age, target, args, optimizer, train_on_gpu)
                        #print("Batch Over")
                        RLoss = RLoss + rloss
                        DLoss = DLoss + dloss
                        #print("RLoss",RLoss)
                        #print("DLoss",DLoss)
                else:
                        rloss=batch_run(model, data, pids, drug, mask, age, target, args, optimizer, train_on_gpu)
                        RLoss = RLoss + rloss

                T.cuda.empty_cache()
                #paramsE=list(model['e'].parameters())
                #paramsG=list(model['g'].parameters())
                ##print("AFTER PARAM",paramsE[0][20],paramsG[8][0][0])   
                #print("one batch complete",batch_idx)

            #print("All batches complete")
            TBatches=TBatches+batch_idx+1
            
        RLoss=RLoss/TBatches
        DLoss=DLoss/TBatches
#         GLoss=GLoss/TBatches   
        print("Train Loss",RLoss)
        if  args.discriminator:
            print("Disc Loss",DLoss)
            trainDLoss.append(DLoss)
        trainLoss.append(RLoss)
        print("validation")

        valid_loss = run_evalFull(args, model, train_on_gpu)
            
        valLoss.append(valid_loss)

        #print("epoch:", epoch, "loss_R:", "%.4f"%RLoss, "loss_G:", "%.4f"%GLoss, "loss_D:", "%.4f"%DLoss)
        #if epoch<1 or epoch >5:
        print(valid_loss)
        valid_loss=valid_loss.cpu()
        if not (np.isnan(valid_loss)):
            early_stopping(valid_loss, model, optimizer, save_path)

        if early_stopping.early_stop:
            print("Early stopping")
            break

#         plot_grad_flow(model['e'].named_parameters())
        plot_grad_flow(model['g'].named_parameters())
#         plot_grad_flow(model['d'].named_parameters())
#         model['g'].printWeights()
#         print("Discriminator Weights: ===============")
#         for name, param in model['d'].named_parameters():
#           print(name, param.grad)
    return trainLoss, valLoss


In [13]:
def main(args):
    ''' Training sequence
    '''
    train_on_gpu = T.cuda.is_available()
    if train_on_gpu:
        print('Training on GPU.')
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    else:
        print('No GPU available, training on CPU.')
        
    #Create Models
    attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
    enc = Encoder(INPUT_DIM, DRUG_DIM, AGE_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
    dec = Decoder(OUTPUT_DIM, AGE_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

    if  args.discriminator:
        model = {
            'g' : Seq2Seq(enc, dec),
            'd' : Discriminator(MAX_SEQ_LEN, D_HID_SIZE)
        }
    else:
        model = {'g' : Seq2Seq(enc, dec)}

    if  args.discriminator:
        optimizer = {
            'g': optim.Adam(model['g'].parameters(), lr=1e-4),
            'd': optim.Adam(model['d'].parameters(), lr=1e-3)
        }
    else:
        optimizer = {
        'g': optim.Adam(model['g'].parameters(), lr=1e-4)
    }
    
    if train_on_gpu:
        model['g'].cuda(0)
        if  args.discriminator:
            model['d'].cuda(0)
        
    if args.resume_training:
        checkpoint = T.load(save_path)
        model['g'].load_state_dict(checkpoint['G_model'])
        if  args.discriminator:
            model['d'].load_state_dict(checkpoint['D_model'])
        optimizer['g'].load_state_dict(checkpoint['G_trainer'])
        if  args.discriminator:
            optimizer['d'].load_state_dict(checkpoint['D_trainer'])
        #Save Updated Model
        
        trainloss, valLoss = run_epoch(args, model, optimizer, train_on_gpu)
        #trainloss, valLoss = run_evalFull(args, model, train_on_gpu)
        
        return trainloss, valLoss
    
    elif args.fine_tune:
        print("================FINE-TUNING===================")
        checkpoint = T.load(save_path)
        model['g'].load_state_dict(checkpoint['G_model'])
        if  args.discriminator:
            model['d'].load_state_dict(checkpoint['D_model'])
        optimizer['g'].load_state_dict(checkpoint['G_trainer'])
        if  args.discriminator:
            optimizer['d'].load_state_dict(checkpoint['D_trainer'])
        #Save Updated Model
        
        trainloss, valLoss = run_epoch(args, model, optimizer, train_on_gpu) 
        
        return trainloss, valLoss
    
    elif args.train:
        print("================PRE-TRAINING===================")
        trainloss, valLoss = run_epoch(args, model, optimizer, train_on_gpu) 
        
        return trainloss, valLoss
 
    elif args.evalPred:
        #load Model
        checkpoint = T.load(save_path)
        model['g'].load_state_dict(checkpoint['G_model'])
        if  args.discriminator:
            model['d'].load_state_dict(checkpoint['D_model'])
        optimizer['g'].load_state_dict(checkpoint['G_trainer'])
        if  args.discriminator:
            optimizer['d'].load_state_dict(checkpoint['D_trainer'])
        oBmi, iBmi, oAge, pids,oAgeIndex, attn, oAttnIdx = pred_test(args, model, train_on_gpu)
        
        return oBmi, iBmi, oAge, pids,oAgeIndex, attn, oAttnIdx
    
    #'epoch': epoch + 1,
    
    
    
        

In [14]:
oBmi, iBmi, oAge, pids,oAgeIndex, attn, oAttnIdx = main(ARGS)
#trainloss, valLoss = main(ARGS)

No GPU available, training on CPU.
idxs torch.Size([354])
idxs torch.Size([250])
idxs torch.Size([372])
idxs torch.Size([273])
idxs torch.Size([331])
idxs torch.Size([278])
idxs torch.Size([299])
idxs torch.Size([267])
idxs torch.Size([320])
idxs torch.Size([319])
idxs torch.Size([437])
idxs torch.Size([281])
idxs torch.Size([257])
idxs torch.Size([299])
idxs torch.Size([326])
idxs torch.Size([283])
idxs torch.Size([371])
idxs torch.Size([170])
RLoss: tensor(0.8495)


In [15]:
import pickle

with open('data/chil/oBmi', 'wb') as fp:
    pickle.dump(oBmi, fp)
    
with open('data/chil/iBmi', 'wb') as fp:
    pickle.dump(iBmi, fp)

with open('data/chil/oAge', 'wb') as fp:
    pickle.dump(oAge, fp)

with open('data/chil/oAgeIndex', 'wb') as fp:
    pickle.dump(oAgeIndex, fp)

with open('data/chil/pids', 'wb') as fp:
    pickle.dump(pids, fp)
    
with open('data/chil/attn', 'wb') as fp:
    pickle.dump(attn, fp)

with open('data/chil/oAttnIdx', 'wb') as fp:
    pickle.dump(oAttnIdx, fp)

In [16]:
# rand_tensor = torch.rand((2, 5, 4, 7))
# rand_tensor

In [17]:
# rand_tensor = rand_tensor.sum(2)
# rand_tensor

In [18]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

In [19]:
input

tensor([[ 0.7969, -0.4043, -0.3568,  0.3643,  1.3026],
        [ 1.2603, -0.1253,  1.6005, -0.0188,  0.2139],
        [-1.5900, -0.0507,  0.2119, -0.7483, -0.8816]], requires_grad=True)

In [20]:
target

tensor([3, 3, 1])

In [21]:
loss = nn.CrossEntropyLoss()
output = loss(input, target)

In [22]:
output

tensor(1.8353, grad_fn=<NllLossBackward>)

In [23]:
ageIndices=torch.tensor(np.array(range(0,40)))
ageIndices=ageIndices.repeat(200)
ageIndices=ageIndices.reshape(200,40)
ageIndices=ageIndices.unsqueeze
ageIndices.shape

AttributeError: 'builtin_function_or_method' object has no attribute 'shape'

In [ ]:
ageIndices[0]

In [ ]:
ageIndices[1]